# Instrument Classification - Part 2 : CNN

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Activation,BatchNormalization

In [ ]:
instr = ['pia','gac']
mix = ['pia-gac']

data_path = '../Data/CQT: 80_12_256/'

In [39]:
def to_categorical(list):
    list_final = []
    for label in list:
        list_final.append(label.split('-'))
    temp_list = [[0]*len(instr)]*(len(list_final))
    for i in range(len(list_final)):
        x = [0]*len(instr)
        x.insert(0,1)
        x.pop()
        for ins in instr:
            if (ins in list_final[i]): 
                temp_list[i] = [sum(x) for x in zip(temp_list[i],x)]
            x.insert(0,x.pop())
    return temp_list

def mix(solo_num,mix_num,):
    first = True
    y_train,y_test = [],[]
    for ins in instr+mix:
        x_train = np.load(data_path+ins+'/x_train_' + ins + '.npy')
        x_test  = np.load('x_test_' + ins + '.npy')
    
        if ins in instr: limit = solo_num
        else: limit = mix_num
        
        for i in range(int(0.9*limit)):
            y_train.append(ins)
        for i in range(int(0.1*limit)):
            y_test.append(ins)
        if (first):
            first = False
            x_train_final = x_train[0:int(0.9*limit)]
            x_test_final = x_test[0:int(0.1*limit)]
        else:
            x_train_final = np.concatenate([x_train_final,x_train[0:int(0.9*limit)]],axis=0)
            x_test_final = np.concatenate([x_test_final, x_test[0:int(0.1*limit)]],axis=0)

    y_train = np.array(to_categorical(y_train))
    y_test = np.array(to_categorical(y_test))

    id1 = np.random.permutation(len(x_train_final))
    id2 = np.random.permutation(len(x_test_final))
    x_train,y_train = x_train_final[id1], y_train[id1]
    x_test,y_test = x_test_final[id2], y_test[id2]
    return (x_train,x_test,y_train,y_test)

In [59]:
solo_num = 10000 #between 0 and 10,000
mix_num  = 8000 #between 0 and 10,000

(x_train,x_test,y_train,y_test) = mix(solo_num,mix_num)

print("Train Size:", x_train.shape)
print("Test  Size:", x_test.shape)

Train Size: (22500, 80, 87, 1)
Test  Size: (2500, 80, 87, 1)


In [ ]:
model = Sequential()

model.add(Conv2D(64,kernel_size=(3,3),padding='same',strides=1,input_shape = x_train[0].shape))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(640,kernel_size=(3,3),padding='same',strides=1))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Dense(6))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
#model = keras.models.load_model('ICmodel_2_mix_big_more.h5')

sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=2,batch_size=32,validation_data=(x_test,y_test))

plt.plot(history.history['acc'], label='Train Accuracy')
plt.plot(history.history['val_acc'], label='Test Accuracy')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy-Loss')
plt.legend()

model.save('ICmodel_3sec.h5')